<a target="_blank" href="https://colab.research.google.com/github/giordamaug/HELP/blob/main/help/notebooks/experiment.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://www.kaggle.com/notebooks/welcome?src=https://github.com/giordamaug/HELP/blob/main/help/notebooks/experiment.ipynb">
  <img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Colab"/>
</a>

# Install HELP from GitHub
Skip this cell if you already have installed HELP.

In [ ]:
!pip install git+https://github.com/giordamaug/HELP.git

# Download the input files
In this cell we download from GitHub repository the label file and the attribute files. Skip this step if you already have these input files locally.

In [ ]:
tissue='Kidney'
!wget https://raw.githubusercontent.com/giordamaug/HELP/main/help/datafinal/{tissue}_HELP.csv
!wget https://raw.githubusercontent.com/giordamaug/HELP/main/help/datafinal/{tissue}_BIO.csv
for i in range(5):
  !wget https://raw.githubusercontent.com/giordamaug/HELP/main/help/datafinal/{tissue}_CCcfs_{i}.csv
!wget https://raw.githubusercontent.com/giordamaug/HELP/main/help/datafinal/{tissue}_EmbN2V_128.csv
#!wget https://raw.githubusercontent.com/giordamaug/HELP/main/help/datafinal/{tissue}_CCBeder.csv
#for i in range(15):
#  !wget https://raw.githubusercontent.com/giordamaug/HELP/main/help/datafinal/{tissue}_BPBeder_{i}.csv

# Run script for experiment
This is the batch script for EG prediction used for the experiments.
the manual page iof EG_prediction.py is the follwing:

In [13]:
!python EG_prediction.py -h

usage: EG_prediction.py [-h] -i <inputfile> [<inputfile> ...]
                        [-X <excludelabels> [<excludelabels> ...]]
                        [-L <labelname>] -l <labelfile> [-A <aliases>]
                        [-b <seed>] [-r <repeat>] [-f <folds>] [-j <jobs>]
                        [-B] [-sf <subfolds>] [-P] [-ba] [-fx]
                        [-n <normalize>] [-o <outfile>] [-s <scorefile>]

PLOS COMPBIO

options:
  -h, --help            show this help message and exit
  -i <inputfile> [<inputfile> ...], --inputfile <inputfile> [<inputfile> ...]
                        input attribute filename list
  -X <excludelabels> [<excludelabels> ...], --excludelabels <excludelabels> [<excludelabels> ...]
                        labels to exclude (default NaN, values any list)
  -L <labelname>, --labelname <labelname>
                        label name (default label)
  -l <labelfile>, --labelfile <labelfile>
                        label filename
  -A <aliases>, --aliases <alias

# E vs NE experiments for Kidney

In [3]:
datapath = "../datafinal"
tissue = "Kidney"                               # or 'Lung'
labelfile = f"{tissue}_HELP.csv"                # label filename
aliases = "-A \"{'aE': 'NE', 'sNE':'NE'}\""     # dictionary for renaming labels before prediction: es. {'oldlabel': 'newlabel'}
#aliases = ""
#excludeflags = "-X aE"                         # label to remove: es. -X aE (for E vs sNE problem)
excludeflags = ""                               
njobs = "-1"                                    # parallelism level: -1 = all cpus, 1 = sequential
sfolds = "4"                                    # dataset subsampling factor: es: 4 for 1:4 ratio of <minority-class>:<majority-class>
nchunks = "1 5 1"                                 # no. of chunks for each input attribute file: es. 1 5 (Bio is one chunk, CCcfs split in 5 chunks)
!python EG_prediction.py -i {datapath}/{tissue}_BIO.csv \
                            {datapath}/{tissue}_CCcfs.csv \
                            {datapath}/{tissue}_EmbN2V_128.csv \
                            -c 1 5 \
                            -l {datapath}/{labelfile} \
                            {aliases} {excludeflags}  \
                            -n std -ba -sf {sfolds} \
                            -j {njobs} -P

METHOD: LGBM	MODE: prob	BALANCE: yes
PROBL: E vs NE
INPUT: Kidney_BIO.csv Kidney_CCcfs.csv
LABEL: Kidney_HELP.csv DISTRIB: E : 1242, NE: 4809
SUBSAMPLE: 1:4
+-------------+-------------------------------+
|             | measure                       |
|-------------+-------------------------------|
| ROC-AUC     | 0.9500±0.0067                 |
| Accuracy    | 0.9038±0.0077                 |
| BA          | 0.8616±0.0123                 |
| Sensitivity | 0.7900±0.0238                 |
| Specificity | 0.9332±0.0078                 |
| MCC         | 0.7110±0.0225                 |
| CM          | [[9812, 2608], [3212, 44878]] |
+-------------+-------------------------------+


In [2]:
!python EG_prediction.py -i {datapath}/{tissue}_BIO.csv {datapath}/{tissue}_CCcfs.csv {datapath}/{tissue}_EmbN2V_128.csv -l {datapath}/{labelfile} {aliases} -n std -ba -sf 1 {excludeflags}  -j {njobs} -P -B

METHOD: LGBM	MODE: prob	BALANCE: yes
PROBL: E vs NE
INPUT: Kidney_BIO.csv Kidney_CCcfs.csv Kidney_EmbN2V_128.csv
LABEL: Kidney_HELP.csv DISTRIB: E : 1242, NE: 1206
SUBSAMPLE: 1:1
+-------------+--------------------------------+
|             | measure                        |
|-------------+--------------------------------|
| ROC-AUC     | 0.9443±0.0092                  |
| Accuracy    | 0.8793±0.0131                  |
| BA          | 0.8791±0.0131                  |
| Sensitivity | 0.8944±0.0181                  |
| Specificity | 0.8638±0.0198                  |
| MCC         | 0.7591±0.0261                  |
| CM          | [[11109, 1311], [1643, 10417]] |
+-------------+--------------------------------+


# E vs sNE experiments for Kidney

In [ ]:
datapath = "../datafinal"
tissue = "Kidney"                               # or 'Lung'
labelfile = f"{tissue}_HELP.csv"                # label filename
aliases = ""                                    # dictionary for renaming labels before prediction: es. {'oldlabel': 'newlabel'}
excludeflags = "-X aE"                          # label to remove: es. -X aE (for E vs sNE problem)
njobs = "-1"                                    # parallelism level: -1 = all cpus, 1 = sequential
sfolds = "4"                                    # dataset subsampling factor: es: 4 for 1:4 ratio of <minority-class>:<majority-class>
nchunks = "1 5 1"                                 # no. of chunks for each input attribute file: es. 1 5 (Bio is one chunk, CCcfs split in 5 chunks)
!python EG_prediction.py -i {datapath}/{tissue}_BIO.csv \
                            {datapath}/{tissue}_CCcfs.csv \
                            {datapath}/{tissue}_EmbN2V_128.csv \
                            -c 1 5 \
                            -l {datapath}/{labelfile} \
                            {aliases} {excludeflags}  \
                            -n std -ba -sf {sfolds} \
                            -j {njobs} -P

In [16]:
!python EG_prediction.py -i {datapath}/{tissue}_BIO.csv {datapath}/{tissue}_CCcfs.csv {datapath}/{tissue}_EmbN2V_128.csv -l {datapath}/{labelfile} {aliases} -ba -sf 2 {excludeflags}  -j {njobs} -P -B

METHOD: LGBM	MODE: prob	BALANCE: yes
PROBL: E vs NE
INPUT: Kidney_BIO.csv Kidney_CCcfs.csv Kidney_EmbN2V_128.csv
LABEL: Kidney_HELP.csv DISTRIB: E : 1242, NE: 2416
SUBSAMPLE: 1:2
+-------------+--------------------------------+
|             | measure                        |
|-------------+--------------------------------|
| ROC-AUC     | 0.9491±0.0063                  |
| Accuracy    | 0.8867±0.0108                  |
| BA          | 0.8753±0.0120                  |
| Sensitivity | 0.8398±0.0202                  |
| Specificity | 0.9108±0.0122                  |
| MCC         | 0.7485±0.0238                  |
| CM          | [[10430, 1990], [2154, 22006]] |
+-------------+--------------------------------+


In [17]:
!python EG_prediction.py -i {datapath}/{tissue}_BIO.csv {datapath}/{tissue}_CCcfs.csv {datapath}/{tissue}_EmbN2V_128.csv -l {datapath}/{labelfile} {aliases} -n std -ba -sf 4 {excludeflags}  -j {njobs} -P -B

METHOD: LGBM	MODE: prob	BALANCE: yes
PROBL: E vs NE
INPUT: Kidney_BIO.csv Kidney_CCcfs.csv Kidney_EmbN2V_128.csv
LABEL: Kidney_HELP.csv DISTRIB: E : 1242, NE: 4809
SUBSAMPLE: 1:4
+-------------+-------------------------------+
|             | measure                       |
|-------------+-------------------------------|
| ROC-AUC     | 0.9528±0.0071                 |
| Accuracy    | 0.9081±0.0081                 |
| BA          | 0.8629±0.0130                 |
| Sensitivity | 0.7862±0.0264                 |
| Specificity | 0.9397±0.0092                 |
| MCC         | 0.7209±0.0238                 |
| CM          | [[9764, 2656], [2902, 45188]] |
+-------------+-------------------------------+


In [18]:
!python EG_prediction.py -i {datapath}/{tissue}_BIO.csv {datapath}/{tissue}_CCcfs.csv {datapath}/{tissue}_EmbN2V_128.csv -l {datapath}/{labelfile} {aliases} -n std  -ba -sf 6 {excludeflags}  -j {njobs} -P -B

METHOD: LGBM	MODE: prob	BALANCE: yes
PROBL: E vs NE
INPUT: Kidney_BIO.csv Kidney_CCcfs.csv Kidney_EmbN2V_128.csv
LABEL: Kidney_HELP.csv DISTRIB: E : 1242, NE: 7223
SUBSAMPLE: 1:6
+-------------+-------------------------------+
|             | measure                       |
|-------------+-------------------------------|
| ROC-AUC     | 0.9557±0.0048                 |
| Accuracy    | 0.9244±0.0056                 |
| BA          | 0.8555±0.0129                 |
| Sensitivity | 0.7580±0.0262                 |
| Specificity | 0.9530±0.0055                 |
| MCC         | 0.7022±0.0214                 |
| CM          | [[9414, 3006], [3395, 68835]] |
+-------------+-------------------------------+


In [19]:
!python EG_prediction.py -i {datapath}/{tissue}_BIO.csv {datapath}/{tissue}_CCcfs.csv {datapath}/{tissue}_EmbN2V_128.csv -l {datapath}/{labelfile} {aliases} -n std -ba -sf 8 {excludeflags}  -j {njobs} -P -B

METHOD: LGBM	MODE: prob	BALANCE: yes
PROBL: E vs NE
INPUT: Kidney_BIO.csv Kidney_CCcfs.csv Kidney_EmbN2V_128.csv
LABEL: Kidney_HELP.csv DISTRIB: E : 1242, NE: 9606
SUBSAMPLE: 1:8
+-------------+-------------------------------+
|             | measure                       |
|-------------+-------------------------------|
| ROC-AUC     | 0.9558±0.0064                 |
| Accuracy    | 0.9335±0.0054                 |
| BA          | 0.8467±0.0137                 |
| Sensitivity | 0.7342±0.0270                 |
| Specificity | 0.9593±0.0050                 |
| MCC         | 0.6794±0.0247                 |
| CM          | [[9119, 3301], [3912, 92148]] |
+-------------+-------------------------------+


In [20]:
!python EG_prediction.py -i {datapath}/{tissue}_BIO.csv {datapath}/{tissue}_CCcfs.csv {datapath}/{tissue}_EmbN2V_128.csv -l {datapath}/{labelfile} {aliases} -n std -ba -sf 10 {excludeflags}  -j {njobs} -P -B

METHOD: LGBM	MODE: prob	BALANCE: yes
PROBL: E vs NE
INPUT: Kidney_BIO.csv Kidney_CCcfs.csv Kidney_EmbN2V_128.csv
LABEL: Kidney_HELP.csv DISTRIB: E : 1242, NE: 12007
SUBSAMPLE: 1:10
+-------------+--------------------------------+
|             | measure                        |
|-------------+--------------------------------|
| ROC-AUC     | 0.9567±0.0047                  |
| Accuracy    | 0.9403±0.0037                  |
| BA          | 0.8421±0.0129                  |
| Sensitivity | 0.7213±0.0262                  |
| Specificity | 0.9630±0.0036                  |
| MCC         | 0.6615±0.0201                  |
| CM          | [[8959, 3461], [4448, 115622]] |
+-------------+--------------------------------+
